# Glider Speed polars analysis notebook

In [352]:
import numpy as np
import math
from numpy.polynomial import Polynomial

print(np.__version__)

# https://www.geeksforgeeks.org/python-implementation-of-polynomial-regression/?ref=lbp
# old vs new API https://numpy.org/doc/1.21/reference/routines.polynomials.html

1.21.1


In [353]:
FILE_TO_LOAD = './lak19-polars-db.json'
#FILE_TO_LOAD = './ventus2-15m-polars-db.json'


In [354]:
# utilities functions & constants

KM_TO_MS = 3.6				# factor to convert km/h in m/s
CONVERT_TO_MS = False		# True if we want to convert speed from km/h to m/s

POLAR_CURVE_START_KM = 70		# in km/h
POLAR_CURVE_END_KM = 230		# in km/h

POLAR_CURVE_START = (POLAR_CURVE_START_KM / KM_TO_MS) if CONVERT_TO_MS else POLAR_CURVE_START_KM
POLAR_CURVE_END = (POLAR_CURVE_END_KM / KM_TO_MS)if CONVERT_TO_MS else POLAR_CURVE_END_KM

POLAR_CURVE_NBR_SAMPLE =  33

MASS_UNITS = { 'wing_loading': 'kg/m2', 'weight' : 'kg'}		# Some units use for polar curves relative to the glider mass
MASS_TXT = { 'wing_loading': 'wing loading', 'weight' : 'weight'}

def polar_to_string(p):
	polar_str = '{}x<sup>2</sup>'.format(round(p.coeffs[0],4))
	polar_str += (' + ' if (p.coeffs[1] >= 0 ) else ' ')
	polar_str += '{}x'.format(round(p.coeffs[1],4))
	polar_str += (' + ' if (p.coeffs[2] >= 0 ) else ' ')
	polar_str += '{}'.format(round(p.coeffs[2],4))
	return polar_str

# to convert speed from km/h to m/s
xaxis_unit = lambda x: x/ (KM_TO_MS if CONVERT_TO_MS else 1)

In [355]:
def _min_sink_rate(polar):
	a = polar.coefficients[0]
	b = polar.coefficients[1]
	c = polar.coefficients[2]

	msr_speed = -b/(2*a) 
	msr_vz = a * np.power(msr_speed,2) +b* msr_speed + c
	msr_ld = -msr_speed/(3.6*msr_vz)

	return msr_speed, msr_vz, msr_ld

def _max_glide_ratio(polar):
	a= polar.coefficients[0]
	b= polar.coefficients[1]
	c= polar.coefficients[2]

	k = b - np.sqrt(4 * a * c)
	mgr_speed = -(b - k) / (2 * a)
	mgr_vz = a * np.power(mgr_speed,2) + b * mgr_speed + c
	mgr_ld = -mgr_speed/(3.6*mgr_vz)

	return mgr_speed, mgr_vz, mgr_ld

In [356]:
# To plot  polar when x&y are 'relevé' by hand
def data4plotly_by_hand(item, mass_key):
	polar_data = dict()

	wl = item[mass_key]
	tn = 'By hand ({}: {} {})'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key])

	polar_data["x"] = item['speed']
	polar_data["y"] = item['sink_rate']
	polar_data["L/D"] = None					# TODO
	polar_data["serie_name"] = tn
	polar_data["polynomial"] = None
	return polar_data

In [357]:
# To plot a polar curve when it is defined by 3 coefficients, y = - (Ax^2 + Bx + C)
def data4plotly_ABC(item, mass_key):
	polar_data = dict()

	wl = item[mass_key]
	tn = 'Model ABC ({}: {} {})'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key])

	polar = np.poly1d([item["A"],item["B"], item["C"] ])
	polar = np.polymul(polar,-1)
	print ('\nModel ABC, polynomial ({}: {} {}) is'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key]))
	print(polar) 

	x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
	# print('\r\nVitesse (km/h)= {}'.format(x_polar))

	y_polar = polar(np.divide(x_polar,100))
	# print('\r\nSink rate (m/s) = {}'.format(y_polar))

	ld = np.divide(np.divide(-x_polar , KM_TO_MS), y_polar)
	# print('\r\nL/D = {}'.format(ld))

	polar_data["x"] = x_polar
	polar_data["y"] = y_polar
	polar_data["L/D"] = ld
	polar_data["serie_name"] = tn
	polar_data["polynomial"] = polar
	polar_data["min_sink_rate"] = _min_sink_rate(polar)
	polar_data["max_glide_ratio"] = _max_glide_ratio(polar)
	return polar_data

In [358]:
# To plot polar curve when it is defined by 3 points (Vi in km/h, Vz in m/s)
def data4plotly_3_points(item, mass_key):
	polar_data = dict()

	speeds = [xaxis_unit(x) for x in item['speed']]
	sink_rate = item['sink_rate']
	wl = item[mass_key]
	tn = 'Model 3-points ({}: {} {})'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key])

	# Fit a polynomial of degree 2 based on the 3 points 
	polar = np.poly1d(np.polyfit(speeds, sink_rate, 2))
	print ('\nModel 3 points, polynomial ({}: {} {}) is'.format(MASS_TXT[mass_key], wl, MASS_UNITS[mass_key]))
	print(polar) 

	# The curve will be the calculated polar using this polynomial, 	
	x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
	y_polar = polar(x_polar)
	# print('\r\nc3_y_polar = {}'.format(y_polar))

	polar_data["x"] = x_polar
	polar_data["y"] = y_polar
	polar_data["L/D"] = None					# TODO
	polar_data["serie_name"] = tn
	polar_data["polynomial"] = polar
	polar_data["min_sink_rate"] = _min_sink_rate(polar)
	polar_data["max_glide_ratio"] = _max_glide_ratio(polar)
	return polar_data

In [359]:

## Polar data import from json file
import json
glider_pdata = None	# glider polars data

def data_for_plotly(item, mass_key):
	if item['method'] == 'by-hand':
		return data4plotly_by_hand(item, mass_key)
	elif item['method'] == 'ABC':
		return data4plotly_ABC(item, mass_key)
	elif item['method'] == '3-points':
		return data4plotly_3_points(item, mass_key)
	else:
		print('Error, method {} is unknow.'.format(item['method']))

# Opening JSON file
with open(FILE_TO_LOAD) as json_file:
	glider_pdata = json.load(json_file)

# Wing loading as unit used for a given polar curve else assume it's the weight
WL_AS_REF_UNIT = 'wing_loading' if (('gp_mass_unit' in glider_pdata.keys()) and (glider_pdata['gp_mass_unit'] == 'wing_loading')) else 'weight'

# Then compute the different curves to plot
polars_ref_data = []
polars_mdl_data = None

print(glider_pdata['specifications'].keys())

for key, value in glider_pdata['specifications'].items():
	if key.startswith('reference'):
		value['polar'] = data_for_plotly(value, WL_AS_REF_UNIT)
		polars_ref_data.append(value)
	elif key.startswith('model'):
		value['polar'] = data_for_plotly(value, WL_AS_REF_UNIT)
		polars_mdl_data = value
	else:
		print('Error, specification {} is unknow !'.format(key))

# print (polars_ref_data)
# print (polars_mdl_data)


dict_keys(['reference-1', 'reference-2', 'reference-3', 'model'])

Model 3 points, polynomial (wing loading: 31.5 kg/m2) is
            2
-0.0001267 x + 0.01687 x - 1.02

Model ABC, polynomial (wing loading: 31.6 kg/m2) is
       2
-1.24 x + 1.61 x - 0.97

Model 3 points, polynomial (wing loading: 30.1 kg/m2) is
            2
-0.0002022 x + 0.03674 x - 2.133

Model 3 points, polynomial (wing loading: 35.7 kg/m2) is
            2
-0.0002167 x + 0.04017 x - 2.45


In [360]:
adjusted_polar = dict()
adjusted_polar['polar_ref'] = {'x': polars_mdl_data['polar']['x'], 'y': polars_mdl_data['polar']['y'], 'wing_loading':  polars_mdl_data['wing_loading'] }
adjusted_polar['polar'] = {
		'x': polars_mdl_data['polar']['x'],
		'y': polars_mdl_data['polar']['y'],
		WL_AS_REF_UNIT:  polars_mdl_data[WL_AS_REF_UNIT],
		'polynomial':  polars_mdl_data['polar']['polynomial'],
		'L/D':  polars_mdl_data['polar']['L/D'],
		'min_sink_rate':  polars_mdl_data['polar']['min_sink_rate'],
		'max_glide_ratio':  polars_mdl_data['polar']['max_glide_ratio'],
	 }

print(adjusted_polar)

{'polar_ref': {'x': array([ 70.,  75.,  80.,  85.,  90.,  95., 100., 105., 110., 115., 120.,
       125., 130., 135., 140., 145., 150., 155., 160., 165., 170., 175.,
       180., 185., 190., 195., 200., 205., 210., 215., 220., 225., 230.]), 'y': array([-0.7       , -0.65625   , -0.62333333, -0.60125   , -0.59      ,
       -0.58958333, -0.6       , -0.62125   , -0.65333333, -0.69625   ,
       -0.75      , -0.81458333, -0.89      , -0.97625   , -1.07333333,
       -1.18125   , -1.3       , -1.42958333, -1.57      , -1.72125   ,
       -1.88333333, -2.05625   , -2.24      , -2.43458333, -2.64      ,
       -2.85625   , -3.08333333, -3.32125   , -3.57      , -3.82958333,
       -4.1       , -4.38125   , -4.67333333]), 'wing_loading': 35.7}, 'polar': {'x': array([ 70.,  75.,  80.,  85.,  90.,  95., 100., 105., 110., 115., 120.,
       125., 130., 135., 140., 145., 150., 155., 160., 165., 170., 175.,
       180., 185., 190., 195., 200., 205., 210., 215., 220., 225., 230.]), 'y': array([-0.

In [361]:
# class for extracting information from the plotly.graph_object Figure()
class TraceInfo:
	def __init__(self, figure):
		self.fig = figure
		self.traces = self.__traces()
		self.number = self.__len__()
		self.names = self.__trace_names()
		self.annotations = self.__annotations_names()

	def __getitem__(self, index):
		return self.traces[index]

	def __len__(self):
		return len(self.traces)

	def __traces(self):
		return self.fig['data']

	def __trace_names(self):
		return [trace['name'] for trace in self.traces]

	def __annotations_names(self):
		return [annotation['name'] for annotation in self.fig['layout']['annotations']]
	
	def trace_index(self, name_to_find, startWith = False):
		idx = 0
		for name in self.names:
			if (not startWith ) and (name == name_to_find):
				return idx
			elif (startWith and name.startswith(name_to_find)):
				return idx
			idx +=1 
		return None
	
	def annotation_index(self, name_to_find):
		idx = 0
		for annotation in self.annotations:
			if annotation.startswith(name_to_find):
				return idx
			idx +=1 
		return None


In [362]:
from scipy.optimize import fsolve

def tangent_horizontal(f,fprime,x):
	roots = fsolve(lambda x: fprime(x), x)
	y = f(roots)
	m = fprime(roots)
	return y - m * (x - roots)

def tangent_at_origin(f, fprime, x):
	roots_estimate = 100,
	roots =fsolve(lambda u: f(u) - fprime(u)*u, roots_estimate)
	return fprime(roots)*x 

def intersection(x, tangent, f, df):
	roots = fsolve(lambda x: tangent(f,df,x) - f(x), x)
	return roots, f(roots)

In [363]:
import plotly.graph_objects as go

def display_tangent_horizontal(polar_polynomial):
	a = polar_polynomial.coefficients[0]
	b = polar_polynomial.coefficients[1]
	c = polar_polynomial.coefficients[2]
	
	f = lambda x: a*np.power(x,2) + b*x + c
	df = lambda x: 2*a*x + b

	tg_x = np.arange(0,250)
	tgh_y = tangent_horizontal(f, df, tg_x)

	x1_int, y1_int = intersection(100,tangent_horizontal,f,df)

	tg_trace = go.Scatter( x=tg_x, y=tgh_y, mode='lines', name = 'Horizontal tangent', line = dict(dash='dash') )
	msr_marker = go.Scatter( x=x1_int, y=y1_int, mode='markers', name = 'Min sink rate', marker_symbol = 'circle',marker_size=10 )
	return tg_trace, msr_marker

def display_tangent_at_origin(polar_polynomial):
	a = polar_polynomial.coefficients[0]
	b = polar_polynomial.coefficients[1]
	c = polar_polynomial.coefficients[2]
	
	f = lambda x: a*np.power(x,2) + b*x + c
	df = lambda x: 2*a*x + b

	tg_x = np.arange(0,250)
	tgao_y = tangent_at_origin(f, df, tg_x)

	x_int, y_int = intersection(100,tangent_at_origin,f,df)

	tg_trace = go.Scatter( x=tg_x, y=tgao_y, mode='lines', name = 'Tangent at (0,0)', line = dict(dash='dash') )
	mgr_marker = go.Scatter( x=x_int, y=y_int, mode='markers', name = 'Max glide ratio', marker_symbol = 'circle',marker_size=10 )

	return tg_trace, mgr_marker

# calculate polar for a specific weight (or wing loading) 
def polarAdjusted4Weight(new_weight, x, y, old_weight):
	new_x = [ vi * math.sqrt(new_weight/old_weight) for vi in x]
	new_y = [ vz * math.sqrt(new_weight/old_weight) for vz in y]

	return new_x, new_y

def update_polar(new_weight, figure):
	weight_ref = adjusted_polar['polar_ref'][WL_AS_REF_UNIT]

	# Update the polar for the new weight
	new_x, new_y = polarAdjusted4Weight(new_weight,adjusted_polar['polar_ref']['x'],adjusted_polar['polar_ref']['y'],weight_ref)
	adjusted_polar['polar']['x'] = new_x
	adjusted_polar['polar']['x'] = new_y

	# update the polynomial
	polar_polynomial = np.poly1d(np.polyfit(new_x, new_y, 2))
	adjusted_polar['polar']['polynomial'] = polar_polynomial

	# update glide ratio
	# TODO

	# update min sink rate and max glide ratio
	adjusted_polar['polar']['min_sink_rate'] = _min_sink_rate(polar_polynomial)
	adjusted_polar['polar']['max_glide_ratio'] = _max_glide_ratio(polar_polynomial)

	# Min sink rate update
	tgt_hz_trace, msr_marker = display_tangent_horizontal(adjusted_polar['polar']['polynomial'])

	# Max glide ratio update
	tgt_ao_trace, mgr__marker = display_tangent_at_origin(adjusted_polar['polar']['polynomial'])

	# Update existing traces
	t = TraceInfo(figure)

	polar_trace_idx = t.trace_index('Adjust',True)
	figure.data[polar_trace_idx].x = new_x
	figure.data[polar_trace_idx].y = new_y

	tgt_hz_trace_idx = t.trace_index('Horizontal tangent')
	figure.data[tgt_hz_trace_idx].x = tgt_hz_trace.x
	figure.data[tgt_hz_trace_idx].y = tgt_hz_trace.y

	tgt_ao_trace_idx = t.trace_index('Tangent at (0,0)')
	figure.data[tgt_ao_trace_idx].x = tgt_ao_trace.x
	figure.data[tgt_ao_trace_idx].y = tgt_ao_trace.y

	msr_marker_idx = t.trace_index('Min sink rate')
	figure.data[msr_marker_idx].x = msr_marker.x
	figure.data[msr_marker_idx].y = msr_marker.y

	mgr_marker_idx = t.trace_index('Max glide ratio')
	figure.data[mgr_marker_idx].x = mgr__marker.x
	figure.data[mgr_marker_idx].y = mgr__marker.y

	# Update annotation min sink rate
	min_sink_rate = adjusted_polar['polar']['min_sink_rate']
	text_annotation = '<b>Min sink rate</b><br> speed: {}km/h<br> Vz: {}m/s<br>L/D: {}'.format(round(min_sink_rate[0],1),round(min_sink_rate[1],2), round(min_sink_rate[2],1) )
	msr_a_idx = t.annotation_index('min-sink-rate')
	figure['layout']['annotations'][msr_a_idx].x = min_sink_rate[0]
	figure['layout']['annotations'][msr_a_idx].y = min_sink_rate[1]
	figure['layout']['annotations'][msr_a_idx].text = text_annotation

	# Update annotation max glide ratio
	max_glide_ratio = adjusted_polar['polar']['max_glide_ratio']
	text_annotation = '<b>Max glide ratio</b><br> speed: {}km/h<br> Vz: {}m/s<br>L/D: {}'.format(round(max_glide_ratio[0],1),round(max_glide_ratio[1],2), round(max_glide_ratio[2],1) )
	mgr_a_idx = t.annotation_index('max-glide-ratio')
	figure['layout']['annotations'][mgr_a_idx].x = max_glide_ratio[0]
	figure['layout']['annotations'][mgr_a_idx].y = max_glide_ratio[1]
	figure['layout']['annotations'][mgr_a_idx].text = text_annotation

	# Update annotation polynomial of the polar curve
	text_annotation = 'Polynomial equation for the polar:<br> {}'.format( polar_to_string(adjusted_polar['polar']['polynomial']))
	polynomial_a_idx = t.annotation_index('polynomial')
	figure['layout']['annotations'][polynomial_a_idx].text = text_annotation

	# return adjusted_polar
	return None


In [364]:
# Display the graph

# https://plotly.com/python/plotly-fundamentals/
# https://plotly.com/python-api-reference/
# https://ipywidgets.readthedocs.io/en/stable/

import plotly.graph_objects as go
from ipywidgets import widgets, Layout, Label
import ipywidgets as iw

traces=[]

# add reference polar curves
for aPolar in polars_ref_data:
	if aPolar['polar'] is not None:
		trace = go.Scatter( x=aPolar['polar']['x'], y=aPolar['polar']['y'], mode='lines', 
			name = '<b>{}</b><br><i>{}</i><br>'.format(aPolar['name'],aPolar['polar']['serie_name']),
			hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', line = dict(dash='dash'))
		traces.append(trace)

# add traces for the model curves, the one we want to modify parameters (weight or wing loading) and interpolate new polar curve
if polars_mdl_data is not None:
	traces.append(go.Scatter( x=polars_mdl_data['polar']['x'], y=polars_mdl_data['polar']['y'], mode='lines', 
		name = '<b>{}</b><br><i>{}</i>'.format(polars_mdl_data['name'],polars_mdl_data['polar']['serie_name']),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', ))
	if WL_AS_REF_UNIT == 'wing_loading':
		traces.append(go.Scatter( x=polars_mdl_data['speed'], y=polars_mdl_data['sink_rate'], mode='markers', name='p1, p2, p3 points', marker_symbol = 'diamond',marker_size=10,
			text=['Point #1', 'Point #2', 'Point #3'], hovertemplate='<extra></extra><b>%{text}</b><br>Speed: %{x}km/h<br>Sink rate: %{y}m/s', hoverinfo='x+y+text'))
	
	# then trace curves that will be able to adjust base on wing-loading
	tgt_hz_trace, msr__marker = display_tangent_horizontal(polars_mdl_data['polar']["polynomial"])
	traces.append(tgt_hz_trace)
	traces.append(msr__marker)

	tgt_ao_trace, mgr__marker = display_tangent_at_origin(polars_mdl_data['polar']["polynomial"])
	traces.append(tgt_ao_trace)
	traces.append(mgr__marker)

	traces.append(go.Scatter( x=adjusted_polar['polar']['x'], y=adjusted_polar['polar']['y'], mode='lines+markers', 
		name = 'Adjusted-v2 ({}: {} {})'.format(MASS_TXT[WL_AS_REF_UNIT], adjusted_polar['polar'][WL_AS_REF_UNIT], MASS_UNITS[WL_AS_REF_UNIT]),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y', ))

layout = go.Layout(
		title_text='<b>{} - Speed polars analysis</b><br><span class="font-size: smaller;">(wing area: {}m2)</span>'.format(glider_pdata['glider'], glider_pdata['wing_area']),
		height=800,
		yaxis=dict(range=[-4, 1]),
		xaxis=dict(range=[0, 250]),
		template='ggplot2',
	)

fig = go.Figure(data=traces, layout=layout)

fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black',  dtick=0.5)
fig.update_yaxes(title_text='Vz (m/s)', title_font=dict(size=14, family='Courier', color='crimson'))

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
fig.update_xaxes(title_text='Vitesse ({})'.format(( 'm/s' if CONVERT_TO_MS else 'km/h')),
	 title_font=dict(size=14, family='Courier', color='crimson'))

if polars_mdl_data is not None:
	text_annotation = 'Polynomial equation for the polar:<br> {}'.format( polar_to_string(polars_mdl_data['polar']['polynomial']))
	fig.add_annotation(
			name = 'polynomial',
			x=xaxis_unit(50), y=0.4, 
			text=text_annotation,
			bgcolor="white",
			borderpad=8,
			bordercolor='gray',
			borderwidth=4,
			showarrow=False,
		)
	
	# Min sink rate annotation
	min_sink_rate = polars_mdl_data['polar']['min_sink_rate']
	text_annotation = '<b>Min sink rate</b><br> speed: {}km/h<br> Vz: {}m/s<br>L/D: {}'.format(round(min_sink_rate[0],1),round(min_sink_rate[1],2), round(min_sink_rate[2],1) )
	# text_annotation = '<b>Min sink rate</b>: speed: {}km/h, Vz: {}m/s, L/D: {}'.format(round(min_sink_rate[0],1),round(min_sink_rate[1],2), round(min_sink_rate[2],1) )
	fig.add_annotation(
			name = 'min-sink-rate',
			# x=xaxis_unit(50), y=0.2, 
			x=min_sink_rate[0], y=min_sink_rate[1], 
			text=text_annotation,
			bgcolor="white", borderpad=8, bordercolor='gray', borderwidth=2,
			showarrow=True,
			yshift=-5,
			ax= -10, ay= 80,
			arrowhead=2,  arrowsize=1, arrowwidth=2,
		)

	# Max glide ratio annotation
	max_gilde_ratio = polars_mdl_data['polar']['max_glide_ratio']
	text_annotation = '<b>Max glide ratio</b><br> speed: {}km/h<br> Vz: {}m/s<br>L/D: {}'.format(round(max_gilde_ratio[0],1),round(max_gilde_ratio[1],2), round(max_gilde_ratio[2],1) )
	fig.add_annotation(
			name = 'max-glide-ratio',
			x=max_gilde_ratio[0], y=max_gilde_ratio[1], 
			text=text_annotation,
			bgcolor="white", borderpad=8, bordercolor='gray', borderwidth=2,
			showarrow=True,
			yshift=5,xshift=5,
			ax= 100, ay= -50,
			arrowhead=2,  arrowsize=1, arrowwidth=2,
		)

fig.update_layout(xaxis_range=[0,max(traces[0].x)*1.2])

# use FigureWiget insead of fig.show() to add controls
# fig.show()
f2 = go.FigureWidget(fig)

# if wing-loading
if WL_AS_REF_UNIT == 'wing_loading':
	ref_init_alue = polars_mdl_data['wing_loading'] 
	ref_unit = MASS_UNITS['wing_loading']
	ref_desc = MASS_TXT['wing_loading']

	weights = widgets.FloatSlider(
		value=polars_mdl_data['wing_loading'],
		min=29,
		max=52,
		step=0.1,
		description= MASS_TXT['wing_loading'],
		disabled=False,
		continuous_update=True,
		# orientation='vertical',
		readout=False,
		readout_format='.1f',
	)
	weight_text = Label(value=f'{weights.value:.1f} '+ref_unit, description='', layout=Layout(width='80px'))
else: 
	ref_init_alue = polars_mdl_data['weight'] 
	ref_unit = MASS_UNITS['weight']
	ref_desc = MASS_TXT['weight']

	weights = widgets.IntSlider(
		value=ref_init_alue,
		min=300,
		max=550,
		step=1,
		description=ref_desc,
		continuous_update=True,
		readout=False 
	)
	weight_text = Label(value=f'{weights.value:d} '+ref_unit, description='', layout=Layout(width='80px'))

container = widgets.HBox(children=[weights,weight_text,], layout=Layout(margin='0px 0px 100px 100px'))

def on_value_change(change):
	if WL_AS_REF_UNIT == 'wing_loading':
		weight_text.value = f'{weights.value:.1f} '+ref_unit
	else:
		weight_text.value = f'{weights.value:d} '+ref_unit
	with f2.batch_update():
		update_polar(weights.value,f2)
		
		t = 'Adjusted ({}: {} {})'.format(ref_desc, weights.value, ref_unit )
		f2.data[-1].name = t

weights.observe(on_value_change, names="value")
widgets.VBox([f2, container])


    'data': [{'hoverinfo': 'x+y',
              'hovertemplate': '<extra></extra…